## New DATASETS 
- Cultural Events
- Restaurants
- Hotels and Airbnbs
- Para la naturaleza and DRNA webpage
- Events

# Concenrs
1) Do I need to spend time on a code to scrape the websites?
2) Since is just for the proyect, focus on finding a static list of datasets (Hotels,Airbnb, Resutarunts and Events)

- I will have to focus on giving a lot of good promts so I can always have the control of the conversation when the user interacts with the chatbot.
- If i dont find a good dataset of hotels and Airbnb, make sure the promts is setup to provide the user a link to booking and airbnb.
- Similar to events and activities, go to "https://app.voyturisteando.com/directorio"
- Provide similar tools that can help him improve the planning.https://mapa.plateapr.com/
- Can can agent help look for nearby spot accounting on the location of the user? 

# Putting the data into DataFrames would be a good approach for several reasons:
 1. Easy data manipulation and analysis
 2. Built-in methods for handling missing values
 3. Efficient filtering and sorting
 4. Simple integration with other data processing libraries
 5. Good for structured data representation

-  We can create DataFrames for each type of data:
    - landmarks_df: to store landmark information
    - municipalities_df: to store municipality data 
    - news_df: to store news articles

- This will make it easier to:
    - Clean and preprocess the data
    - Extract relevant features
    - Prepare data for vector embeddings
    - Track metadata

## LETS LOAD DATASETS





In [ ]:
import pandas as pd
#Load Municipality
municipalities_df = pd.read_csv('data/municipalities_extracted/Proccessed/Best/municipalities_structured.csv')


#lOAD landmarks DATASET
landmarks_df_corr = pd.read_csv('Last_LANDMARK_020925/processed_landmarks_gpt3_images_STRUCTURED_FILL_NANS.csv')


In [ ]:
municipalities_df.head(3)

In [ ]:
landmarks_df_corr.head(2)

## Let Initalize a VectorStore and add the embedded data to it. 

- The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

- For now, let's us PINECONE. An alternative can be CHROMA

# PineCone Upload and Retrieval function to acccess the data from the VectorDB
 - This one has the landmark database on Pinecone

# Steps:
1) Initialize Pinecone Index
2) Initialize Retrieval Model
3) Upload embeded data to VectorDB

# 1) Initialize Pinecone  and retriever model

In [ ]:
import os
from pinecone import Pinecone
from pinecone import ServerlessSpec

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'

In [ ]:

# configure client
pc = Pinecone(api_key=api_key)

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

index_name = "pr-landmarks-struct-csv-try1" #give your index a meaningful name

import time

# check if index already exists (it shouldn't if this is first time)
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if the abstractive-question-answering index exists
if index_name not in existing_indexes:
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=1024,  # matches the embedding dimension
        metric='cosine',
        spec=spec
    )

# connect to abstractive-question-answering index we created
index = pc.Index(index_name)

# view index stats
index.describe_index_stats()


In [ ]:
##-------------- LOAD EMBEDING MODEL --------------##
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load the retriever model from huggingface model hub
#retriever = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base', device=device)#


# Option 1: Multilingual model (good for Spanish/English content)
retriever = SentenceTransformer('intfloat/multilingual-e5-large', device=device)

# Option 2: Travel/location specialized model
#retriever = SentenceTransformer('BAAI/bge-large-en-v1.5', device=device)

retriever

In [ ]:
index.describe_index_stats()

# 2) Prpepera data for Vector data base 

In [ ]:
from sentence_transformers import SentenceTransformer
import pinecone
from typing import List, Dict
import json
import os
from tqdm.auto import tqdm
import pandas as pd

def clean_metadata_value(value, field_name):
    """Clean metadata values with meaningful placeholders based on field type"""
    if pd.isna(value) or (isinstance(value, str) and value.strip() == '') or value is None:
        # Custom placeholders based on field type
        placeholders = {
            'visit_duration': "Visit duration varies", 
            'hours': "Contact location for current hours",
            'admission': "Contact location for current prices",
            'content': "No detailed description available",
            'key_features': ["Information to be updated"],
            'primary_category': "Uncategorized",
            'secondary_category': "General", 
            'chatbot_tags': ["needs_update"],
            'town': "Location to be verified",
            'latitude': 0.0,
            'longitude': 0.0,
            'website': "No website listed",
            'direction': "N/A",
            'images': []
        }
        
        return placeholders.get(field_name, "Information not available")
    
    if isinstance(value, (int, float)):
        return value
    elif isinstance(value, list):
        if not value:
            return ["Information to be updated"]
        return [str(item) for item in value if pd.notna(item)]
    elif isinstance(value, dict):
        return {k: clean_metadata_value(v, k) for k, v in value.items()}
    return str(value)

def prepare_landmarks_batch(batch_df, retriever):
    """Prepare a batch of landmark data for Pinecone upload"""
    vectors = []
    embeddings = []
    
    for idx, row in batch_df.iterrows():
        # Use the pre-formatted text_for_embedding
        embedding = retriever.encode(row['text_for_embedding']).tolist()
        embeddings.append(embedding)
        
        metadata = {
            'type': 'landmark',
            'name': clean_metadata_value(row['landmark_name'], 'name'),
            'coordinates': clean_metadata_value(row['coordinates'], 'coordinates'),
            'location': clean_metadata_value(row['location'], 'location'),
            'details': clean_metadata_value(row['details'], 'details'),
            'content': clean_metadata_value(row['content'], 'content'),
            'images': clean_metadata_value(row['images'], 'images'),
            'metadata': clean_metadata_value(row['metadata'], 'metadata'),
            'data_completeness': 'partial' if any(
                pd.isna(row[field]) for field in ['landmark_name', 'coordinates', 'location', 'content']
            ) else 'complete'
        }
        
        vectors.append({
            'id': f"landmark_{idx}",
            'values': embedding,
            'metadata': metadata
        })
    
    return vectors, embeddings

def prepare_municipalities_batch(batch_df, retriever):
    """Prepare a batch of municipality data for Pinecone upload"""
    vectors = []
    embeddings = []
    
    for idx, row in batch_df.iterrows():
        # Use the pre-formatted text_for_embedding
        embedding = retriever.encode(row['text_for_embedding']).tolist()
        embeddings.append(embedding)
        
        metadata = {
            'type': 'municipality',
            'name': clean_metadata_value(row['municipality_name'], 'name'),
            'latitude': clean_metadata_value(row['latitude'], 'latitude'),
            'longitude': clean_metadata_value(row['longitude'], 'longitude'),
            'summary': clean_metadata_value(row['summary'], 'summary'),
            'image_urls': clean_metadata_value(row['image_urls'], 'images'),
            'has_images': clean_metadata_value(row['has_images'], 'has_images'),
            'coordinates_valid': clean_metadata_value(row['coordinates_valid'], 'coordinates_valid'),
            'google_maps_url': clean_metadata_value(row['google_maps_url'], 'google_maps_url'),
            'data_completeness': 'partial' if any(
                pd.isna(row[field]) for field in ['municipality_name', 'latitude', 'longitude', 'summary']
            ) else 'complete'
        }
        
        vectors.append({
            'id': f"municipality_{idx}",
            'values': embedding,
            'metadata': metadata
        })
    
    return vectors, embeddings

# 3) Upload Embedded datasets to VectorDB - PineCone

In [ ]:
# Initialize model and Pinecone
print("Initializing...")

# Lists to store all embeddings and track errors
all_landmark_embeddings = []
all_municipality_embeddings = []
failed_uploads = {'landmarks': [], 'municipalities': []}

# Upload landmarks in batches with progress bar
print("Uploading landmark vectors...")
batch_size = 64  # You can adjust this based on your needs
for i in tqdm(range(0, len(landmarks_df_corr), batch_size)):
    i_end = min(i + batch_size, len(landmarks_df_corr))
    batch = landmarks_df_corr.iloc[i:i_end]
    
    try:
        vectors, embeddings = prepare_landmarks_batch(batch, retriever)
        all_landmark_embeddings.extend(embeddings)
        
        # Verify vectors before upload
        if vectors and len(vectors) > 0:
            index.upsert(vectors=vectors)
        else:
            print(f"\nWarning: Empty vectors generated for landmarks batch {i//batch_size}")
            
    except Exception as e:
        print(f"\nError with landmarks batch {i//batch_size}: {str(e)}")
        failed_uploads['landmarks'].append({
            'batch_index': i//batch_size,
            'error': str(e),
            'rows': list(batch.index)
        })
        continue

# Upload municipalities in batches with progress bar
print("\nUploading municipality vectors...")
for i in tqdm(range(0, len(municipalities_df), batch_size)):
    i_end = min(i + batch_size, len(municipalities_df))
    batch = municipalities_df.iloc[i:i_end]
    
    try:
        vectors, embeddings = prepare_municipalities_batch(batch, retriever)
        all_municipality_embeddings.extend(embeddings)
        
        # Verify vectors before upload
        if vectors and len(vectors) > 0:
            index.upsert(vectors=vectors)
        else:
            print(f"\nWarning: Empty vectors generated for municipalities batch {i//batch_size}")
            
    except Exception as e:
        print(f"\nError with municipalities batch {i//batch_size}: {str(e)}")
        failed_uploads['municipalities'].append({
            'batch_index': i//batch_size,
            'error': str(e),
            'rows': list(batch.index)
        })
        continue



In [ ]:
# Print upload statistics
print("\nUpload Statistics:")
print(f"Landmarks uploaded: {len(all_landmark_embeddings)}")
print(f"Municipalities uploaded: {len(all_municipality_embeddings)}")
print(f"Failed landmark batches: {len(failed_uploads['landmarks'])}")
print(f"Failed municipality batches: {len(failed_uploads['municipalities'])}")

# Print index statistics
print("\nVector Database Statistics:")
index_stats = index.describe_index_stats()
print(index_stats)

# Save failed uploads for later investigation if needed
#if any(failed_uploads.values()):
#    with open('failed_uploads.json', 'w') as f:
#        json.dump(failed_uploads, f, indent=2)
#    print("\nFailed uploads have been saved to 'failed_uploads.json'")

# 4) Retreiver and Example

In [ ]:
# Test query function
def search_locations(query: str, filter_params: Dict = None, top_k: int = 3):
    """Search for locations with handling for placeholder values"""
    # Generate query embedding
    xq = retriever.encode(query).tolist()
    
    # Perform search
    results = index.query(
        vector=xq,
        filter=filter_params,
        top_k=top_k,
        include_metadata=True
    )
    
    # Format results to indicate missing information
    formatted_results = []
    for match in results['matches']:
        result = {
            'name': match['metadata']['name'],
            'score': match['score'],
            'data_completeness': match['metadata'].get('data_completeness', 'unknown'),
            'details': {}
        }
        
        # Add available information
        for key, value in match['metadata'].items():
            if key not in ['name', 'data_completeness']:
                if isinstance(value, str) and 'not available' in value.lower():
                    result['details'][key] = f"⚠️ {value}"
                else:
                    result['details'][key] = value
                    
        formatted_results.append(result)
    
    return formatted_results



# Test the search
test_queriess = [
    "Show me beaches in Vieques",
    "What museums are in San Juan?",
    "Historical sites in Ponce",
    "Tell me places to hike in the East Coast of Puerto Rico?",
    "Best places to eat at the west side of the island?",
    "What is the most prestige University and School in Puerto Rico?"
]

for query in test_queriess:
    print(f"\nQuery: {query}")
    resultss = search_locations(query)
    for r in resultss:
        print(f"\nName: {r['name']} (Score: {r['score']:.2f})")
        print(f"Data completeness: {r['data_completeness']}")
        for k, v in r['details'].items():
            if isinstance(v, str) and '⚠️' in v:
                print(f"{k}: {v}")
            else:
                print(f"{k}: {v}")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import Document

# Initialize LLM
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    openai_api_key=os.getenv('OPENAI_API_KEY')
)
# Create prompt template for location search

LOCATION_SEARCH_PROMPT = PromptTemplate(
    input_variables=["query", "search_results"],
    template="""You are a helpful travel assistant for Puerto Rico. Use the search results to provide detailed information about locations.

    Question: {query}
    
    Search Results:
    {search_results}
    
    Instructions:
    1. Always include exact coordinates when available (e.g., "18.0983°N, 65.4714°W")
    2. Mention practical information (hours, admission, etc.) when relevant
    3. If information is missing, specify what details are not available
    4. Always mention the town/municipality where places are located
    5. Include relevant image URLs if available
    6. Specify the direction (N,S,E,W,Center) of the location
    
    Format your response to be engaging and informative, using emojis for better readability:
    🏷️ [Location Name]
    📍 [Coordinates/Municipality]
    🔗 [URL/Images]
    ℹ️ [Description]
    ⏰ [Hours/Admission]
    💡 [Travel Tips]"""
)

# Function to format search results
def format_search_results(results):
    """Format search results for the prompt"""
    formatted_results = []
    for match in results['matches']:
        result = {
            'name': match['metadata']['name'],
            'score': match['score'],
            'details': {}
        }
        
        # Add available information
        for key, value in match['metadata'].items():
            if key not in ['name', 'data_completeness']:
                if isinstance(value, str) and 'not available' in value.lower():
                    result['details'][key] = f"⚠️ {value}"
                else:
                    result['details'][key] = value
        
        # Format the result as string
        result_str = [f"\n🏷️ {result['name']} (Relevance: {result['score']:.2f})"]
        for k, v in result['details'].items():
            result_str.append(f"{k}: {v}")
        
        formatted_results.append("\n".join(result_str))
    
    return "\n---\n".join(formatted_results)

# Create the search and response chain
async def search_and_respond(query: str, retriever, index, llm, top_k: int = 3):
    """Search locations and generate response"""
    try:
        # Generate query embedding
        xq = retriever.encode(query).tolist()
        
        # Perform search
        results = index.query(
            vector=xq,
            top_k=top_k,
            include_metadata=True
        )
        
        # Format results
        formatted_results = format_search_results(results)
        
        # Create chain
        location_chain = LLMChain(
            llm=llm,
            prompt=LOCATION_SEARCH_PROMPT
        )
        
        # Generate response
        response = await location_chain.arun(
            query=query,
            search_results=formatted_results
        )
        
        return response
        
    except Exception as e:
        print(f"Error in search_and_respond: {str(e)}")
        return "I encountered an error while searching. Please try again."

# Initialize the chain
location_search_chain = LLMChain(
    llm=llm,
    prompt=LOCATION_SEARCH_PROMPT
)

# Test the chain
async def test_search_chain():
    test_queries = [
        "Show me beaches in Vieques",
        "What museums are in San Juan?",
        "Historical sites in Ponce",
        "Tell me places to hike in the East Coast of Puerto Rico?",
        "Best places to eat at the west side of the island?"
    ]
    
    for query in test_queries:
        print(f"\nQuery: {query}")
        print("-" * 50)
        response = await search_and_respond(
            query=query,
            retriever=retriever,
            index=index,
            llm=llm
        )
        print(response)
        print("=" * 80)

# Run the test
await test_search_chain()

# Multi- Query Retieval Model

# Now that I have the data in the Vector DataBase i would like to implement a Mulit-query with langchain. 

- A Multi-query retrieval approach is highly recommended for your travel planner because:
    1) Better Query Understanding:
        - Generates multiple variations of the user's question
        - Captures different aspects of the same query
        - Handles different ways people might ask about the same thing
    2) Improved Coverage:
        - Finds relevant results even if they don't exactly match the original query
        - Especially useful for travel queries that can be phrased many ways
        - Better handles complex queries like "family-friendly beaches with food nearby"

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
import os

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-large'
)

# Initialize LLM
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    openai_api_key=os.getenv('OPENAI_API_KEY')
)

# Create Pinecone vectorstore wrapper
vectorstore = Pinecone(
    index=index,  # Your existing Pinecone index
    embedding=embeddings,
    text_key="content"  # The metadata field containing the text content
)

# Create prompt template
prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI travel assistant for Puerto Rico tourism. 
    Generate multiple search queries to find relevant places based on this question.
    Consider all aspects: location, category, features, and practical details.

    Original question: {question}

    Generate 3 different search queries that cover:                            
    - Location and nearby areas
    - Type of attraction/activity
    - Specific features or amenities
    - Practical considerations (hours, admission, duration)

    Make queries specific to Puerto Rico tourism and landmarks."""
)


# Create the multi-query retriever
retriever2 = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    llm=llm,
    prompt=prompt
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

question = "Where can I find waterfalls and hiking trails near San Juan??"

docs = retriever2.get_relevant_documents(query=question)
len(docs)




In [ ]:
docs

# Add Promts with Instruccion and how it will response. 

In [ ]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import Document

# Improved prompt template that emphasizes using metadata
QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful travel assistant for Puerto Rico. Use the context content, direction, longitude, latitude, images AND metadata to answer questions about locations in Puerto Rico. Always include specific details like coordinates, hours, features, and images when available.

    Context information:
    {contexts}

    Question: {query}
    
    Instructions:
    1. Always include exact coordinates when available (e.g., "18.0983°N, 65.4714°W")
    2. Mention practical information (hours, admission, etc.) when relevant
    3. If information is missing, specify what details are not available
    4. Always mention the town/municipality where places are located
    5. Include relevant image URLs if available
    6. Specify the direction (N,S,E,W,Center) of the location
    
    Format coordinates as "XX.XXXX°N, XX.XXXX°W" when available.
    
    Answer the question naturally and conversationally."""
)

async def generate_response(query: str, contexts: list):
    """Generate response using both content and metadata"""
    # Format contexts to include metadata
    formatted_contexts = []
    for doc in contexts:
        # Extract metadata and parse string-formatted dictionaries
        metadata = doc.metadata
        
        try:
            # Parse coordinates from string to dict if needed
            coordinates = metadata.get('coordinates', '{}')
            if isinstance(coordinates, str):
                coordinates = eval(coordinates)
            latitude = coordinates.get('latitude', 'Unknown')
            longitude = coordinates.get('longitude', 'Unknown')
            
            # Parse location from string to dict if needed
            location = metadata.get('location', '{}')
            if isinstance(location, str):
                location = eval(location)
            town = location.get('town', 'Unknown')
            direction = location.get('direction', 'Unknown')
            
            # Parse details from string to dict if needed
            details = metadata.get('details', '{}')
            if isinstance(details, str):
                details = eval(details)
            primary_category = details.get('primary_category', 'Unknown')
            secondary_category = details.get('secondary_category', 'Unknown')
            hours = details.get('hours', 'Hours not available')
            admission = details.get('admission', 'Admission info not available')
            
            # Create formatted context string
            context_str = f"Location: {metadata.get('name', 'Unknown')}\n"
            context_str += f"Town: {town}\n"
            context_str += f"Coordinates: {latitude}°N, {longitude}°W\n"
            context_str += f"Direction: {direction}\n"
            context_str += f"Category: {primary_category} - {secondary_category}\n"
            context_str += f"Description: {doc.page_content}\n"
            context_str += f"Hours: {hours}\n"
            context_str += f"Admission: {admission}\n"
            
            # Handle images
            images = metadata.get('images', '[]')
            if isinstance(images, str):
                images = eval(images)
            if images and isinstance(images, list):
                image_urls = [img.get('url', '') for img in images if isinstance(img, dict) and 'url' in img]
                context_str += f"Images: {', '.join(image_urls)}\n"
            else:
                context_str += "Images: No images available\n"
            
            formatted_contexts.append(context_str)
            
        except Exception as e:
            print(f"Error processing metadata: {str(e)}")
            # Add basic context if metadata parsing fails
            context_str = f"Location: {metadata.get('name', 'Unknown')}\n"
            context_str += f"Description: {doc.page_content}\n"
            formatted_contexts.append(context_str)
    
    # Join all contexts
    contexts_text = "\n---\n".join(formatted_contexts)
    
    # Generate response
    response = await qa_chain.arun(
        query=query,
        contexts=contexts_text
    )
    
    return response


# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)


In [ ]:
# Test function that shows both retrieval and response
async def test_qa_with_details(query: str):
    """Test QA with detailed output"""
    print(f"Query: {query}\n")
    
    # Get documents from retriever
    docs = retriever2.get_relevant_documents(query)
    
    print("Retrieved Contexts:")
    for i, doc in enumerate(docs, 1):
        print(f"\nDocument {i}:")
        print(f"Content: {doc.page_content[:200]}...")
        print("Metadata:")
        for k, v in doc.metadata.items():
            if v is not None and v != "":
                print(f"  {k}: {v}")
    
    # Generate response
    response = await generate_response(query, docs)
    print("\nGenerated Response:")
    print(response)

# Example usage
test_queries = [
    "Can you provide me coordinates of the best beaches in Vieques?",
    "What are some family-friendly beaches in Vieques and their facilities?",
    "Tell me about snorkeling spots in the west coast of Puerto Rico and their locations"
]

for query in test_queries:
    print("\n" + "="*80)
    await test_qa_with_details(query)

#

# LANGCHAINS FOR CHATBOT


In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
import logging

# Disable langchain logging
logging.getLogger('langchain').setLevel(logging.ERROR)
logging.getLogger('langchain_community').setLevel(logging.ERROR)

# Enhanced prompt templates with more specific guidance
# Change the variable name to match what's used in the class
DATE_VALIDATION_PROMPT = PromptTemplate(
    input_variables=["date_input"],
    template="""As a Puerto Rico travel expert, validate if {date_input} is a valid future travel date.
    
    Consider:
    1. Date must be in the future
    2. Date should be within next 12 months
    3. Consider Puerto Rico's peak/off-peak seasons
    
    For summer months (May-August):
    - Expect sunny conditions with high temperatures (85-95°F)
    - Light afternoon showers are common
    - Important to stay hydrated and use sun protection
    - Best to plan outdoor activities for morning hours
    
    For winter months (September-February): 
    - Monitor weather conditions for potential floods
    - Check river water levels before water activities
    - Be aware of high wave conditions at beaches
    - Strong winds may affect outdoor plans
    
    If valid, provide relevant seasonal weather guidance.
    If invalid, explain why and ask for a new date.
    
    Format response as:
    VALID: [formatted_date] | [season_info] | [weather_notes]
    or 
    INVALID: [reason]"""
)

location_preference_prompt = PromptTemplate(
    input_variables=["location", "travel_date"],
    template="""Based on the user's location preference: {location}
    Planned travel date: {travel_date}
    
    As a Puerto Rico expert:
    1. Validate if this is a valid Puerto Rico region/area
    2. Consider seasonal factors for {travel_date}
    3. Suggest nearby areas if the location is too specific/limited
    
    Format response as:
    VALID: [location_name] | [nearby_areas] | [seasonal_notes]
    or
    INVALID: [reason] | [suggested_alternatives]"""
)

# Simplified interest analysis prompt to reduce token usage
interest_analysis_prompt = PromptTemplate(
    input_variables=["interests", "location", "travel_date"],
    template="""For {location} on {travel_date}, find attractions matching: {interests}
    
    Create a focused search query combining:
    1. Main interest type
    2. Location specifics
    3. Key activity preferences
    
    Format: QUERY: [interest]|[location]|[activity]"""
)

# Simplified recommendation prompt to reduce token usage
recommendation_prompt = PromptTemplate(
    input_variables=["retrieved_docs", "interests", "location", "travel_date"],
    template="""Based on:
    Location: {location}
    Date: {travel_date}
    Interests: {interests}
    
    Provide 3 best matching recommendations with:
    - Name and brief description
    - Key practical details
    - Local tips
    
    End with:
    "Would you like to:
    1. 🔍 Get more details
    2. ➕ Add to itinerary
    3. 🔄 See other options
    4. 🗺️ Explore another area"
    """
)

class PuertoRicoTravelPlanner:
    def __init__(self, llm, retriever, qa_chain):
        self.date_chain = LLMChain(
            llm=llm,
            prompt=DATE_VALIDATION_PROMPT,
            verbose=False,
            output_key="date_validation"
        )
        
        self.location_chain = LLMChain(
            llm=llm,
            prompt=location_preference_prompt,
            output_key="location_validation"
        )
        
        self.interest_chain = LLMChain(
            llm=llm,
            prompt=interest_analysis_prompt,
            output_key="search_query"
        )
        
        self.recommendation_chain = LLMChain(
            llm=llm,
            prompt=recommendation_prompt,
            output_key="recommendations"
        )
        
        self.retriever = retriever
        
        # Use ConversationBufferMemory with reduced history
        self.retrieval_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=ConversationBufferMemory(
                memory_key="chat_history",
                return_messages=True,
                k=3  # Only keep last 3 exchanges
            )
        )
        
        self.current_state = {
            "travel_date": None,
            "location": None,
            "interests": [],
            "selected_places": []
        }

    async def process_date(self, date_input):
        """Process and validate travel date"""
        try:
            result = await self.date_chain.ainvoke({"date_input": date_input})
            if "date_validation" in result:
                if result["date_validation"].startswith("VALID"):
                    date_info = result["date_validation"].split("|")
                    self.current_state["travel_date"] = date_info[0].replace("VALID:", "").strip()
                return result["date_validation"]
            return "INVALID: Unable to process date"
        except Exception as e:
            return f"INVALID: {str(e)}"

    async def process_location(self, location_input):
        """Process and validate location preference"""
        try:
            # Limit to 3 most relevant documents
            docs = self.retriever.get_relevant_documents(f"Tell me about {location_input} in Puerto Rico")[:3]
            
            if docs:
                location_info = await generate_response(
                    f"Is {location_input} a valid location in Puerto Rico? Answer only VALID or INVALID with brief reason",
                    docs
                )
                
                if "VALID" in location_info.upper():
                    self.current_state["location"] = location_input
                    return f"VALID: {location_input} | {' '.join([doc.metadata.get('town', '') for doc in docs if doc.metadata.get('town')])}"
                else:
                    return f"INVALID: Could not validate {location_input} as a location in Puerto Rico"
            else:
                return f"INVALID: Could not find information about {location_input}"
        except Exception as e:
            return f"INVALID: Error processing location - {str(e)}"

    async def process_interests(self, interests_input):
        """Process user interests and generate recommendations"""
        try:
            # Generate focused search query
            query_result = await self.interest_chain.ainvoke({
                "interests": interests_input,
                "location": self.current_state["location"],
                "travel_date": self.current_state["travel_date"]
            })
            
            # Get top 3 relevant documents
            search_query = query_result["search_query"].split("QUERY:")[1].strip()
            retrieved_docs = self.retriever.get_relevant_documents(search_query)[:3]
            
            # Generate concise recommendations
            recommendations = await self.recommendation_chain.ainvoke({
                "retrieved_docs": retrieved_docs,
                "interests": interests_input,
                "location": self.current_state["location"],
                "travel_date": self.current_state["travel_date"]
            })
            
            return recommendations["recommendations"]
        except Exception as e:
            return f"Error generating recommendations: {str(e)}"

    async def get_location_details(self, location_name):
        """Get detailed information about a specific location"""
        query = f"Brief overview of {location_name} in {self.current_state['location']}"
        result = await self.retrieval_chain.ainvoke({"question": query})
        return result["answer"]

    async def add_to_itinerary(self, location_name):
        """Add a location to the user's itinerary"""
        if location_name not in self.current_state["selected_places"]:
            self.current_state["selected_places"].append(location_name)
            return f"Added {location_name} to your itinerary! 📝"
        return f"{location_name} is already in your itinerary!"

    def get_itinerary(self):
        """Return the current itinerary"""
        if not self.current_state["selected_places"]:
            return "Your itinerary is empty! Let's add some places to visit! 📝"
        
        itinerary = "Here's your current itinerary:\n"
        for i, place in enumerate(self.current_state["selected_places"], 1):
            itinerary += f"{i}. {place}\n"
        return itinerary

async def initialize_travel_planner():
    """Initialize the travel planner with required components"""
    return PuertoRicoTravelPlanner(
        llm=llm,
        retriever=retriever2,
        qa_chain=qa_chain
    )

async def handle_conversation():
    """Main conversation handler"""
    planner = await initialize_travel_planner()
    
    welcome_message = """
    ¡Bienvenidos to the Puerto Rico Travel Guide! 🌴
    I'm your personal travel assistant, ready to help you plan an unforgettable trip to Puerto Rico.
    
    When are you planning to visit? (e.g., December 2024)
    """
    print(welcome_message)
    
    state = "GET_DATE"
    while True:
        try:
            user_input = input("You: ").strip()
            
            if not user_input:
                print("\nBot: Please enter a response.")
                continue
                
            if user_input.lower() in ['quit', 'exit', 'bye', 'adiós']:
                if planner.current_state["selected_places"]:
                    print("\nBefore you go, here's your final itinerary:")
                    print(planner.get_itinerary())
                print("\nBot: ¡Hasta luego! Have a great trip! 🌴✈️")
                break
            
            if state == "GET_DATE":
                date_validation = await planner.process_date(user_input)
                print(f"\nBot: {date_validation}")
                
                if date_validation.startswith("VALID"):
                    state = "GET_LOCATION"
                    print("\nBot: Great! Which area of Puerto Rico would you like to explore?")
                continue
                
            if state == "GET_LOCATION":
                location_validation = await planner.process_location(user_input)
                print(f"\nBot: {location_validation}")
                
                if location_validation.startswith("VALID"):
                    state = "GET_INTERESTS"
                    print("\nBot: Perfect! What kind of activities or places interest you? (e.g., beaches, hiking, history)")
                continue
                
            if state == "GET_INTERESTS":
                recommendations = await planner.process_interests(user_input)
                print(f"\nBot: {recommendations}")
                state = "RECOMMENDATIONS"
                continue
                
            if state == "RECOMMENDATIONS":
                if user_input.startswith("1"):
                    print("\nBot: Which place would you like to know more about?")
                    place_name = input("You: ").strip()
                    details = await planner.get_location_details(place_name)
                    print(f"\nBot: {details}")
                    
                elif user_input.startswith("2"):
                    print("\nBot: Which place would you like to add to your itinerary?")
                    place_name = input("You: ").strip()
                    result = await planner.add_to_itinerary(place_name)
                    print(f"\nBot: {result}")
                    
                elif user_input.startswith("3"):
                    print("\nBot: Let me find some alternative suggestions. What specific type of places would you like to see?")
                    state = "GET_INTERESTS"
                    
                elif user_input.startswith("4"):
                    state = "GET_LOCATION"
                    print("\nBot: Which other area would you like to explore?")
                    
                else:
                    print("\nBot: I didn't understand that choice. Please select 1, 2, 3, or 4.")
                continue
                
        except Exception as e:
            print(f"\nBot: I encountered an error: {str(e)}")
            print("Let's try that again!")
            continue

# Initialize and run the conversation
if __name__ == "__main__":
    await handle_conversation()

# LANGCHAINS FOR CHATBOT - More Interactive chat 

In [ ]:
from typing import Dict, List, Optional
from datetime import datetime
import asyncio
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

from dateutil import parser
from datetime import datetime, timedelta

# Update the interest analysis prompt to be more explicit
INTEREST_ANALYSIS_PROMPT = PromptTemplate(
    input_variables=["interests"],
    template="""Analyze these travel interests for Puerto Rico: {interests}

    You must respond in this exact format with the | symbol:
    CATEGORIES: [comma-separated list of matching categories] | KEYWORDS: [key search terms]

    Available categories:
    1. Nature/Outdoor (beaches, hiking, rainforest, etc.)
    2. Cultural/Historical (museums, architecture, etc.)
    3. Adventure/Sports (surfing, diving, etc.)
    4. Food/Dining (restaurants, food tours, etc.)
    5. Entertainment (nightlife, shows, etc.)

    Example response:
    CATEGORIES: Nature/Outdoor, Adventure/Sports | KEYWORDS: beaches, surfing, water activities

    Remember to always include the | symbol between CATEGORIES and KEYWORDS."""
)

# Update the date validation prompt with current date awareness
DATE_VALIDATION_PROMPT = PromptTemplate(
    input_variables=["date_input", "current_date"],
    template="""As a Puerto Rico travel expert, validate if {date_input} is a valid future travel date.
    
    Current date: {current_date}
    
    Consider these seasons in Puerto Rico:
    1. High Season (Mid-December to Mid-April):
       - Peak tourist season
       - Dry season with less rainfall
       - Average temperatures 75-85°F
       - Perfect for beach activities and outdoor exploration
       - Higher prices and more crowded
    
    2. Shoulder Season (April to June):
       - Moderate tourist traffic
       - Increasing humidity
       - Average temperatures 80-90°F
       - Good for rainforest visits and water activities
       - Better prices on accommodations
    
    3. Low Season (July to November):
       - Hurricane season
       - Frequent afternoon showers
       - Average temperatures 85-95°F
       - Best prices and fewer tourists
       - Need to monitor weather forecasts
       - Many indoor alternatives available
    
    Rules for validation:
    1. Date must be after {current_date}
    2. Format any date input into a clear date
    3. Consider hurricane season risks (July-November)
    
    Format response as:
    VALID: [formatted_date] | [season_type] | [weather_expectations] | [travel_tips]
    or 
    INVALID: [reason] | [suggestion]"""
)

class PuertoRicoTravelBot:
    def __init__(self, llm, qa_chain, retriever):
        """Initialize the travel bot with required components."""
        self.llm = llm
        self.qa_chain = qa_chain
        self.retriever = retriever
        
        # Initialize conversation state
        self.state = {
            "travel_dates": None,
            "interests": [],
            "selected_locations": [],
            "current_topic": "greeting",
            "waiting_for_input": False,
            "last_input": None  # Add this to store the last input
        }
        
        # Initialize chain prompts
        self._init_chains()
        
        # Conversation memory
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )

    def _init_chains(self):
        """Initialize the various prompt chains."""
        # Date validation chain
        self.date_chain = LLMChain(
            llm=self.llm,
            prompt=DATE_VALIDATION_PROMPT
        )
        
        # Interest analysis chain
        self.interest_chain = LLMChain(
            llm=self.llm,
            prompt=PromptTemplate(
                input_variables=["interests"],
                template="""Analyze these travel interests: {interests}
                Categorize them into:
                1. Nature/Outdoor
                2. Cultural/Historical
                3. Adventure/Sports
                4. Food/Dining
                5. Entertainment
                
                Return as:
                CATEGORIES: [list] | KEYWORDS: [key terms for search]"""
            )
        )

    async def start_conversation(self) -> str:
        """Initialize the conversation with a welcome message."""
        welcome_message = """
        ¡Bienvenidos! 🌴 I'm your Puerto Rico Travel Assistant.
        
        I'll help you plan an amazing trip to Puerto Rico based on your interests.
        I can provide information about landmarks, activities, and help you create
        a personalized list of places to visit.
        
        To get started, when are you planning to visit? 
        (For example: "December 2025" or "Next summer")
        """
        self.state["current_topic"] = "get_dates"
        self.state["waiting_for_input"] = True
        return welcome_message
    
    async def _handle_exit(self) -> str:
        """Handle conversation exit."""
        if self.state["selected_locations"]:
            locations = "\n".join(f"- {loc}" for loc in self.state["selected_locations"])
            return f"""
            Thanks for letting me help plan your trip! Here's your final list of places to visit:
            
            {locations}
            
            ¡Buen viaje! 🌴✈️
            """
        return "¡Hasta luego! Feel free to come back when you're ready to plan your Puerto Rico adventure! 🌴"


    async def process_input(self, user_input: str) -> str:
        """Process user input based on current conversation state."""
        if not user_input:
            return "Please provide some input."
            
        # Store the input
        self.state["last_input"] = user_input
            
        # Handle exit commands
        if user_input.lower() in ['exit', 'bye', 'goodbye', 'adiós', 'adios']:
            return await self._handle_exit()

        # Process based on current topic
        if self.state["current_topic"] == "get_dates":
            return await self._handle_dates(user_input)
        elif self.state["current_topic"] == "get_interests":
            return await self._handle_interests(user_input)
        elif self.state["current_topic"] == "suggestions":
            return await self._handle_suggestions(user_input)
        elif self.state["current_topic"] == "questions":
            return await self._handle_questions(user_input)
        
        # Default fallback
        return "I'm not sure how to help with that. Let's focus on planning your trip!"

    async def chat(self):
        """Start and manage the conversation."""
        try:
            # Start conversation
            response = await self.start_conversation()
            print("\nBot:", response)
            
            while True:
                try:
                    # Get user input
                    user_input = input("\nYou: ").strip()
                    
                    # Skip empty inputs
                    if not user_input:
                        continue
                    
                    # Process input and get response
                    response = await self.process_input(user_input)
                    
                    # Only print valid responses
                    if response:
                        print(f"\nBot: {response}")
                        
                        # Check for conversation end
                        if response.endswith("🌴✈️") or response.endswith("🌴"):
                            break  # Use break instead of return
                    
                except Exception as e:
                    print(f"\nError occurred: {str(e)}")
                    print("Let's try again...")
                    
        except Exception as e:
            print(f"\nFatal error: {str(e)}")
            print("Bot conversation ended unexpectedly.")

    async def _handle_dates(self, date_input: str) -> str:
        """Process and validate travel dates."""
        try:
            # Get current date and max future date
            current_date = datetime.now()
            max_future_date = current_date + timedelta(days=365)
            
            # Handle relative dates first
            parsed_date = None
            lower_input = date_input.lower()
            
            if "next" in lower_input:
                if "summer" in lower_input:
                    # Set to June 1st of next summer
                    year = current_date.year + (1 if current_date.month >= 6 else 0)
                    parsed_date = datetime(year, 6, 1)
                elif "winter" in lower_input:
                    # Set to December 1st of next winter
                    year = current_date.year + (1 if current_date.month >= 12 else 0)
                    parsed_date = datetime(year, 12, 1)
                elif "spring" in lower_input:
                    # Set to March 1st of next spring
                    year = current_date.year + (1 if current_date.month >= 3 else 0)
                    parsed_date = datetime(year, 3, 1)
                elif "fall" in lower_input:
                    # Set to September 1st of next fall
                    year = current_date.year + (1 if current_date.month >= 9 else 0)
                    parsed_date = datetime(year, 9, 1)
                elif "month" in lower_input:
                    parsed_date = current_date + timedelta(days=30)
                elif "week" in lower_input:
                    parsed_date = current_date + timedelta(days=7)
            elif "in" in lower_input and "months" in lower_input:
                # Extract number of months
                try:
                    num_months = int(''.join(filter(str.isdigit, lower_input)))
                    parsed_date = current_date + timedelta(days=30 * num_months)
                except ValueError:
                    return "Please specify a clear number of months, like 'in 3 months'"
            
            # If not a relative date, try parsing it normally
            if parsed_date is None:
                try:
                    parsed_date = parser.parse(date_input, fuzzy=True)
                except:
                    return f"""
                    I couldn't understand that date format.
                    Today is {datetime.now().strftime('%B %d, %Y')}.
                    Please try a different format.
                    Error: {str(e)}
                    """

            # Explicit date validation
            if parsed_date < current_date:
                return f"""
                INVALID: The date {parsed_date.strftime('%B %Y')} is in the past. 
                Today is {current_date.strftime('%B %d, %Y')}. 
                Please provide a future date.
                """
            
            if parsed_date > max_future_date:
                return f"""
                INVALID: The date {parsed_date.strftime('%B %Y')} is too far in the future. 
                Please choose a date before {max_future_date.strftime('%B %Y')}.
                """
            
            # Format dates for the chain
            formatted_date = parsed_date.strftime("%B %Y")
            current_date_str = current_date.strftime("%B %d, %Y")
            
            # Get season and weather info from LLM
            result = await self.date_chain.arun(
                date_input=formatted_date,
                current_date=current_date_str
            )
            
            if result.startswith("VALID"):
                self.state["travel_dates"] = formatted_date
                self.state["current_topic"] = "get_interests"
                return f"""
                {result}
                
                Great! Now, tell me what interests you most about Puerto Rico:
                - Nature and beaches? 🏖️
                - History and culture? 🏛️
                - Adventure and sports? 🏄‍♂️
                - Food and dining? 🍽️
                - Or something else?
                
                Feel free to mention multiple interests!
                """
            
            return f"""
            {result.replace('INVALID:', '')}
            Today is {current_date_str}. Please provide a valid future date.
            """
                
        except Exception as e:
            return f"""
            An error occurred while processing your date.
            Today is {datetime.now().strftime('%B %d, %Y')}.
            Please try a different format.
            Error: {str(e)}
            """

    async def _handle_interests(self, interests: str) -> str:
        """Process user interests and generate initial suggestions."""
        try:
            # Use the stored last input if interests is empty
            if not interests and self.state["last_input"]:
                interests = self.state["last_input"]
            
            # 1. Analyze interests using interest_chain
            analysis = await self.interest_chain.arun(interests=interests)
            
            if "|" not in analysis:
                categories = analysis if analysis.startswith("CATEGORIES:") else f"CATEGORIES: {analysis}"
                keywords = "KEYWORDS: general"
                analysis = f"{categories} | {keywords}"
            
            categories, keywords = analysis.split("|", 1)
            
            # Store interests
            self.state["interests"] = [
                cat.strip() 
                for cat in categories.replace("CATEGORIES:", "").strip().split(",")
                if cat.strip()
            ]
            
            # 2. Generate search query based on interests and travel dates
            season_info = "high season" if "High Season" in self.state.get("travel_dates", "") else \
                        "shoulder season" if "Shoulder Season" in self.state.get("travel_dates", "") else \
                        "low season"
            
            base_query = f"""Find attractions in Puerto Rico matching these interests: {interests}.
                        Travel date: {self.state['travel_dates']} ({season_info})
                        Categories: {', '.join(self.state['interests'])}"""
            
            # 3. Use qa_chain to generate multiple search queries
            docs = self.retriever.get_relevant_documents(base_query)
            
            # 4. Use generate_response to get detailed context
            detailed_response = await generate_response(base_query, docs)
            
            # 5. Use location_search_chain for specific recommendations
            location_response = await search_and_respond(
                query=base_query,
                retriever=self.retriever,
                index=self.index,
                llm=self.llm,
                top_k=3
            )
            
            # 6. Combine both responses
            self.state["current_topic"] = "suggestions"
            return f"""
            Based on your interests in {', '.join(self.state['interests'])}, here are my recommendations:
            
            🌟 Overview of Available Activities:
            {detailed_response}
            
            📍 Specific Places to Visit:
            {location_response}
            
            Would you like to:
            1. 🔍 Get more details about any of these places
            2. ➕ Add places to your visit list
            3. 🔄 See different suggestions
            4. ❓ Ask any other questions
            
            Just let me know what you'd like to do!
            """
                
        except Exception as e:
            print(f"Debug - Error in _handle_interests: {str(e)}")
            self.state["current_topic"] = "suggestions"
            
            # Fallback to simpler search
            try:
                docs = self.retriever.get_relevant_documents(interests)
                fallback_response = await generate_response(interests, docs[:3])
                
                return f"""
                Here are some suggestions based on your interests:
                
                {fallback_response}
                
                Would you like to:
                1. 🔍 Get more details about any of these places
                2. ➕ Add places to your visit list
                3. 🔄 See different suggestions
                4. ❓ Ask any other questions
                """
            except Exception as e2:
                print(f"Debug - Fallback error: {str(e2)}")
                return "I encountered an error while searching. Please try again or rephrase your interests."

    async def _handle_suggestions(self, user_input: str) -> str:
        """Handle user response to suggestions."""
        try:
            # Use stored last input if user_input is empty
            if not user_input and self.state["last_input"]:
                user_input = self.state["last_input"]
                
            input_lower = user_input.lower().strip()
            
            # Handle adding locations to list
            if input_lower.startswith('2') or 'add' in input_lower or any(word in input_lower for word in ['list', 'save', 'include']):
                # Extract and add location name
                location_name = None
                
                # Common location indicators
                location_indicators = ['beach','river','waterfall','lake','park', 'museum', 'forest', 'cave', 'mountain', 'restaurant', 'plaza']
                
                # Try to extract location name from input
                words = input_lower.split()
                for i, word in enumerate(words):
                    # Check if word is or contains a location indicator
                    if any(indicator in word for indicator in location_indicators):
                        # Look for a proper noun before or after the indicator
                        if i > 0:  # Check word before
                            location_name = words[i-1].title() + " " + word.title()
                        else:  # Use next word
                            location_name = word.title()
                            if i+1 < len(words):
                                location_name += " " + words[i+1].title()
                        break
                
                if location_name:
                    self.state["selected_locations"].append(location_name)
                    return f"""
                    Great! I've added {location_name} to your list! 📝
                    
                    What else interests you about Puerto Rico? You can:
                    - Tell me about other activities you'd like to do
                    - Ask about different types of places
                    - Get recommendations for food, culture, or nature
                    
                    Or choose from:
                    1. 🔍 Get details about places
                    2. 🔄 See more suggestions
                    3. 📋 View your current list
                    4. ✈️ Finish planning
                    """
                else:
                    return "Which place would you like to add? Please include the name."
                    
            # Handle requests for more activities/interests
            elif any(word in input_lower for word in ['activities', 'things to do', 'what else', 'other activities', 'interested in']):
                self.state["current_topic"] = "get_interests"
                return """
                What other activities interest you? You can choose from:
                - Nature and beaches? 🏖️
                - History and culture? 🏛️
                - Adventure and sports? 🏄‍♂️
                - Food and dining? 🍽️
                - Entertainment? 🎭
                
                Feel free to mention multiple interests!
                """
                    
            # Handle requests for more details
            elif input_lower.startswith('1') or any(word in input_lower for word in ['detail', 'more', 'about', 'tell']):
                self.state["current_topic"] = "questions"
                
                # Try to extract location from the question
                location_words = []
                for word in input_lower.split():
                    # Capitalize first letter of each word for proper nouns
                    if word not in ['more', 'about', 'tell', 'me', 'details', 'information']:
                        location_words.append(word.capitalize())
                
                if location_words:
                    location = ' '.join(location_words)
                    return f"What would you like to know about {location}?"
                else:
                    return "Which place would you like to know more about?"
                    
            # Handle requests for different suggestions
            elif input_lower.startswith('3') or any(word in input_lower for word in ['different', 'other', 'more suggestion', 'else']):
                interests = ", ".join(self.state["interests"])
                docs = self.retriever.get_relevant_documents(interests)
                new_suggestions = await generate_response(interests, docs[3:6])
                
                return f"""
                Here are some alternative suggestions:
                
                {new_suggestions}
                
                Would you like to:
                1. 🔍 Get more details about any of these places
                2. ➕ Add places to your visit list
                3. 🔄 See different suggestions
                4. 📋 View your current list
                """
                
            # Handle viewing current list
            elif input_lower.startswith('4') or any(word in input_lower for word in ['view', 'list', 'show', 'selected']):
                if self.state["selected_locations"]:
                    locations = "\n".join(f"- {loc}" for loc in self.state["selected_locations"])
                    return f"""
                    Here's your current list of places to visit:
                    
                    {locations}
                    
                    Would you like to:
                    1. 🔍 Get details about any of these places
                    2. ➕ Add another place
                    3. 🔄 See more suggestions
                    4. ❌ Remove a place from the list
                    """
                else:
                    return """
                    Your list is currently empty. Would you like to:
                    1. 🔍 Get details about places to visit
                    2. ➕ Add a place to your list
                    3. 🔄 See more suggestions
                    4. ❓ Ask other questions
                    """
                    
            else:
                # Try to understand the intent
                if any(word in input_lower for word in ['where', 'what', 'how', 'when', 'why', 'can', 'is']):
                    self.state["current_topic"] = "questions"
                    return await self._handle_questions(user_input)
                
                return """I'm not sure what you'd like to do. You can:
                1. 🔍 Get more details about places
                2. ➕ Add places to your list
                3. 🔄 See different suggestions
                4. 📋 View your current list
                
                Just tell me what you'd like to do!"""
                
        except Exception as e:
            print(f"Debug - Error in _handle_suggestions: {str(e)}")
            return "I encountered an error. Please try again with a clearer request."

    async def _handle_questions(self, question: str) -> str:
        """Handle user questions using both chains."""
        try:
            # Use stored last input if question is empty
            if not question and self.state["last_input"]:
                question = self.state["last_input"]
                
            # Get relevant documents
            docs = self.retriever.get_relevant_documents(question)
            
            # Format documents for qa_chain
            contexts = []
            for doc in docs:
                # Extract metadata
                metadata = doc.metadata
                
                # Format context with metadata
                context_str = f"Location: {metadata.get('name', 'Unknown')}\n"
                context_str += f"Description: {doc.page_content}\n"
                
                # Add coordinates if available
                if 'coordinates' in metadata:
                    coords = metadata['coordinates']
                    if isinstance(coords, str):
                        coords = eval(coords)
                    context_str += f"Coordinates: {coords.get('latitude', 'Unknown')}°N, {coords.get('longitude', 'Unknown')}°W\n"
                
                # Add other metadata
                for key, value in metadata.items():
                    if key not in ['name', 'coordinates']:
                        context_str += f"{key}: {value}\n"
                
                contexts.append(context_str)
            
            # Join contexts
            contexts_text = "\n---\n".join(contexts)
            
            # Get detailed response using qa_chain
            response = await self.qa_chain.arun(
                query=question,
                contexts=contexts_text
            )
            
            return f"""
            {response}
            
            Would you like to:
            1. 🔍 Ask another question
            2. ➕ Add this place to your visit list
            3. 🔄 See more suggestions
            4. 📋 View your current list
            """
            
        except Exception as e:
            print(f"Debug - Error in _handle_questions: {str(e)}")
            return "I encountered an error. Please try asking your question in a different way."

    async def chat(self):
        """Start and manage the conversation."""
        try:
            # Start conversation
            response = await self.start_conversation()
            print("\nBot:", response)
            
            while True:
                try:
                    # Get user input
                    user_input = input("\nYou: ").strip()
                    
                    # Skip empty inputs
                    if not user_input:
                        continue
                    
                    # Process input and get response
                    response = await self.process_input(user_input)
                    
                    # Only print valid responses
                    if response:
                        print(f"\nBot: {response}")
                        
                        # Check for conversation end
                        if response.endswith("🌴✈️") or response.endswith("🌴"):
                            return  # Exit the function immediately
                    
                except Exception as e:
                    print(f"\nError occurred: {str(e)}")
                    print("Let's try again...")
                    
        except Exception as e:
            print(f"\nFatal error: {str(e)}")
            print("Bot conversation ended unexpectedly.")


# test dates and BOT

In [ ]:
# Update your bot's date handling
bot = PuertoRicoTravelBot(llm=llm, qa_chain=qa_chain, retriever=retriever)
bot.date_chain = LLMChain(llm=llm, prompt=DATE_VALIDATION_PROMPT)

# Test different date formats
test_dates = [
    "December 2025",
    "next summer",
    "in 3 months",
    "12/25/2024",
    "Christmas 2024",
    "next week"
]

async def test_date_handling():
    for date in test_dates:
        print(f"\nTesting date: {date}")
        response = await bot._handle_dates(date)
        print(f"Response: {response}")

# Run the test
await test_date_handling()

In [ ]:
import asyncio
from langchain.chat_models import ChatOpenAI
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Initialize the bot
bot = PuertoRicoTravelBot(
    llm=llm,  # Your existing LLM
    qa_chain=qa_chain,  # Your existing qa_chain
    retriever=retriever2  # Your existing retriever
)

# Add the index to the bot instance
bot.index = index  # Your existing Pinecone index

# Start the chat
await bot.chat()

# NEW CHATBOT

In [ ]:
# Update the date validation prompt with current date awareness
DATE_VALIDATION_PROMPT = PromptTemplate(
    input_variables=["date_input", "current_date"],
    template="""As a Puerto Rico travel expert, validate if {date_input} is a valid future travel date.
    
    Current date: {current_date}
    
    Consider these seasons in Puerto Rico:
    1. High Season (Mid-December to Mid-April):
       - Peak tourist season
       - Dry season with less rainfall
       - Average temperatures 75-85°F
       - Perfect for beach activities and outdoor exploration
       - Higher prices and more crowded
    
    2. Shoulder Season (April to June):
       - Moderate tourist traffic
       - Increasing humidity
       - Average temperatures 80-90°F
       - Good for rainforest visits and water activities
       - Better prices on accommodations
    
    3. Low Season (July to November):
       - Hurricane season
       - Frequent afternoon showers
       - Average temperatures 85-95°F
       - Best prices and fewer tourists
       - Need to monitor weather forecasts
       - Many indoor alternatives available
    
    Rules for validation:
    1. Date must be after {current_date}
    2. Format any date input into a clear date
    3. Consider hurricane season risks (July-November)
    
    Format response as:
    VALID: [formatted_date] | [season_type] | [weather_expectations] | [travel_tips]
    or 
    INVALID: [reason] | [suggestion]"""
)

# Update the interest analysis prompt to be more explicit
INTEREST_ANALYSIS_PROMPT = PromptTemplate(
    input_variables=["interests"],
    template="""Analyze these travel interests for Puerto Rico: {interests}

    You must respond in this exact format with the | symbol:
    CATEGORIES: [comma-separated list of matching categories] | KEYWORDS: [key search terms]

    Available categories:
    1. Nature/Outdoor (beaches, hiking, rainforest, etc.)
    2. Cultural/Historical (museums, architecture, etc.)
    3. Adventure/Sports (surfing, diving, etc.)
    4. Food/Dining (restaurants, food tours, etc.)
    5. Entertainment (nightlife, shows, etc.)

    Example response:
    CATEGORIES: Nature/Outdoor, Adventure/Sports | KEYWORDS: beaches, surfing, water activities

    Remember to always include the | symbol between CATEGORIES and KEYWORDS."""
)


In [ ]:
class SimplePRTravelBot:
    def __init__(self, llm, qa_chain, retriever):
        """Initialize bot with core components."""
        self.llm = llm
        self.qa_chain = qa_chain
        self.retriever = retriever
        
        # State management
        self.state = {
            "travel_dates": None,
            "interests": [],
            "itinerary": [],
            "current_step": "greeting"
        }
        
        # Initialize chains
        self.date_chain = LLMChain(llm=llm, prompt=DATE_VALIDATION_PROMPT)
        self.interest_chain = LLMChain(llm=llm, prompt=INTEREST_ANALYSIS_PROMPT)

    async def start_chat(self):
        """Begin the conversation."""
        welcome = """
        ¡Bienvenidos! 🌴 I'm your Puerto Rico Travel Assistant.
        
        I'll help you plan an amazing trip to Puerto Rico based on your interests.
        To get started, when are you planning to visit? 
        (Example: "December 2025", "next summer", "in 3 months")
        """
        print("\nBot:", welcome)
        
        while True:
            try:
                user_input = input("\nYou: ").strip()
                if not user_input:
                    continue
                    
                if user_input.lower() in ['exit', 'bye', 'goodbye', 'quit']:
                    await self._show_final_itinerary()
                    break
                    
                response = await self._process_input(user_input)
                if response:
                    print("\nBot:", response)
                    
            except Exception as e:
                print(f"\nError: {str(e)}")
                print("Let's try again...")

    async def _process_input(self, user_input: str) -> str:
            """Process user input based on current conversation state."""
            
            if self.state["current_step"] == "greeting":
                # Handle initial date input
                date_response = await self._handle_dates(user_input)
                if date_response.startswith("VALID"):
                    self.state["current_step"] = "get_interests"
                    return f"""
                    Perfect! You're planning to visit in {self.state['travel_dates']}.
                    
                    Tell me what interests you most about Puerto Rico:
                    - Nature and beaches? 🏖️
                    - History and culture? 🏛️
                    - Adventure and sports? 🏄‍♂️
                    - Food and dining? 🍽️
                    - Or something else?
                    
                    Feel free to mention multiple interests!
                    """
                return date_response
                
            elif self.state["current_step"] == "get_interests":
                # Process interests and generate suggestions
                suggestions = await self._handle_interests(user_input)
                self.state["current_step"] = "suggestions"
                return suggestions
                
            elif self.state["current_step"] == "suggestions":
                # Handle user response to suggestions
                if "add" in user_input.lower() or "list" in user_input.lower():
                    return await self._add_to_itinerary(user_input)
                elif "question" in user_input.lower() or "?" in user_input:
                    self.state["current_step"] = "questions"
                    return await self._handle_questions(user_input)
                elif "more" in user_input.lower() or "other" in user_input.lower():
                    self.state["current_step"] = "get_interests"
                    return "What other aspects of Puerto Rico interest you?"
                else:
                    return """
                    I can help you:
                    - Add places to your itinerary
                    - Answer questions about locations
                    - Show more suggestions
                    - View your current itinerary
                    
                    What would you like to do?
                    """
                    
            elif self.state["current_step"] == "questions":
                response = await self._handle_questions(user_input)
                self.state["current_step"] = "suggestions"
                return f"""
                {response}
                
                Would you like to:
                1. Add this place to your itinerary
                2. Ask another question
                3. See more suggestions
                4. View your current itinerary
                """

    async def _handle_interests(self, interests: str) -> str:
            """Process interests and generate suggestions."""
            try:
                # Analyze interests
                analysis = await self.interest_chain.arun(interests=interests)
                categories = analysis.split("|")[0].replace("CATEGORIES:", "").strip()
                
                # Store interests
                self.state["interests"].extend([
                    cat.strip() 
                    for cat in categories.split(",")
                    if cat.strip() not in self.state["interests"]
                ])
                
                # Generate query and get suggestions
                query = f"Find attractions in Puerto Rico for someone interested in {categories}"
                docs = self.retriever.get_relevant_documents(query)
                
                suggestions = []
                for doc in docs[:3]:  # Limit to top 3 suggestions
                    name = doc.metadata.get('name', 'Unknown location')
                    desc = doc.page_content[:200] + "..."  # Truncate long descriptions
                    suggestions.append(f"🏷️ {name}\n{desc}")
                
                return f"""
                Based on your interest in {categories}, here are some suggestions:
                
                {"".join(suggestions)}
                
                Would you like to:
                1. Add any of these places to your itinerary
                2. Ask questions about these places
                3. See more suggestions
                4. View your current itinerary
                """
                
            except Exception as e:
                print(f"Error in interests: {str(e)}")
                return "I had trouble processing that. Could you rephrase your interests?"   

    async def _add_to_itinerary(self, user_input: str) -> str:
        """Add a location to the itinerary."""
        # Extract location name from input
        words = user_input.lower().split()
        location_indicators = ['beach', 'park', 'museum', 'restaurant', 'forest', 'cave']
        
        location = None
        for i, word in enumerate(words):
            if any(indicator in word for indicator in location_indicators):
                if i > 0:
                    location = f"{words[i-1]} {word}".title()
                else:
                    location = word.title()
                break
        
        if location:
            if location not in self.state["itinerary"]:
                self.state["itinerary"].append(location)
                return f"""
                Added {location} to your itinerary! 📝
                
                Would you like to:
                1. See more suggestions
                2. Ask questions about other places
                3. View your current itinerary
                4. Add another place
                """
            else:
                return f"{location} is already in your itinerary. Would you like to add something else?"
        
        return "Which place would you like to add? Please include the location name."

    async def _show_final_itinerary(self) -> str:
        """Display final itinerary and end conversation."""
        if not self.state["itinerary"]:
            print("\nBot: Thanks for chatting! Come back when you're ready to plan your trip! 🌴")
            return
            
        itinerary = "\n".join(f"- {place}" for place in self.state["itinerary"])
        print(f"""
        Bot: Here's your Puerto Rico travel itinerary:
        
        Dates: {self.state['travel_dates']}
        
        Places to visit:
        {itinerary}
        
        ¡Buen viaje! 🌴✈️
        """)

In [ ]:
# Initialize the bot
bot = SimplePRTravelBot(
    llm=llm,  # Your existing LLM
    qa_chain=qa_chain,  # Your existing QA chain
    retriever=retriever  # Your existing retriever
)

# Start the conversation
await bot.start_chat()